# Script d'utilisation de l'API Légifrance
Ce script rend possible l'utilisation de l'API Légifrance, disponible via PISTE. Nos requêtes permettent de constituer une base de données comprenant les nominations en cabinet ministériel sur l'année 2024, et visent à démontrer la faisabilité de ce projet. 

Un travail de grande ampleur ayant déjà été réalisé par Nathann Cohen dans le cadre du projet Jorfsearch, nous employons néanmoins sa base de données dans notre analyse principale afin d'avoir une plus grande plage historique (1990-2024).

## 1/ Mise en place
Il nous faut dans un premier temps obtenir notre token : celui-ci s'obtient grâce aux identifiants générés sur le portail PISTE lorsqu'on a accepté les CGU de l'API Légifrance.

**Étapes à suivre pour obtenir un token**
- Créer un compte PISTE : [https://piste.gouv.fr/component/apiportal/registration]
- Une fois connecté, trouver dans le menu horizontal API -> Consentements CGU API : accepter les CGU pour l'API Légifrance
- Raccorder l'API à votre application sandbox : Applications -> sandbox, cliquer sur le bouton modifier l'application, accéder à la page de consentement, sélectionner Légifrance et valider. Appliquer les modifications. 
- Trouver les identifiants Oauth : Applications -> sélectionner l'application souhaitée -> trouver la section Oauth. Les renseigner en tant que client_id et client_secret dans le script suivant afin d'obtenir un identifiant.

In [6]:
import requests
import pandas as pd

# Prompt the user to input client_id and client_secret : nous ne voulons pas hardcoded nos identifiants
client_id = input("Enter your client_id: ")  # client_id generated by PISTE
client_secret = input("Enter your client_secret: ")  # client_secret generated by PISTE

url = "https://sandbox-oauth.piste.gouv.fr/api/oauth/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(url, data=payload, headers=headers)

# Store the access token in a variable
access_token = None
if response.status_code == 200:
    access_token = response.json().get("access_token")
else:
    print("Error:", response.status_code, response.text)


## 2/ Accès à la documentation Swagger
Afin de comprendre la structure des requêtes possibles, nous accédons à la documentation Swagger.

In [21]:
import json
import requests

# Charger la documentation Swagger depuis une URL
url = "https://github.com/user-attachments/files/17714249/Legifrance.json"
response = requests.get(url)
swagger_doc = response.json()

# Afficher des informations générales
print("Titre:", swagger_doc['info']['title'])
print("Version:", swagger_doc['info']['version'])
print("Description:", swagger_doc['info'].get('description', 'Pas de description disponible'))

# Parcourir les chemins (endpoints)
print("\nEndpoints disponibles :")
for path, methods in swagger_doc['paths'].items():
    print(f"\nPath : {path}")
    for method, details in methods.items():
        print(f"  Méthode : {method.upper()}")
        print("    Description :", details.get('description', 'Pas de description disponible'))

        # Afficher les paramètres de chaque méthode
        if 'parameters' in details:
            print("    Paramètres :")
            for param in details['parameters']:
                param_type = param.get('type', 'inconnu')  # Définit le type à "inconnu" si la clé "type" n'existe pas
                print(f"      - {param['name']} (type: {param_type}) - {'Obligatoire' if param.get('required') else 'Optionnel'}")

        # Afficher les réponses possibles
        print("    Réponses :")
        for status_code, response in details['responses'].items():
            print(f"      - Code {status_code}: {response.get('description', 'Pas de description')}")


Titre: Légifrance
Version: 2.4.2
Description: 
Afin de faciliter la réutilisation des données juridiques, la DILA met à disposition du public une API de Légifrance.
Ce document a pour objectif de décrire les méthodes de l'API Légifrance, présentées à l'aide du framework Swagger.
Vous pouvez également télécharger une [documentation complémentaire](https://www.legifrance.gouv.fr/contenu/Media/Files/pied-de-page/description-des-tris-et-filtres-de-l-api.xlsx) ainsi que des [exemples d’utilisation](https://www.legifrance.gouv.fr/contenu/Media/Files/pied-de-page/exemples-d-utilisation-de-l-api.docx) de l'API.

Cet accès vous permet d’expérimenter l'API sur un environnement de test et de vous familiariser avec les différentes méthodes.

À noter :
- L'environnement de test peut être indisponible.
- Les appels sont limités par quotas.

Les mises à jour correctives et évolutives sont réalisées dans un premier temps sur cet environnement.
L'API mise à disposition correspond à celle utilisée par l

## 3/ Requêtes pour les prénoms au JORF
Notre objectif est d'avoir pour l'année 2024 l'ensemble des nominations. Pour ce faire, nous procédons en deux temps : 
- Nous identifions d'abord l'ensemble des arrêtés de nomination à l'aide de leur identifiant (NOR).
- Pour chacun de ces NOR, nous identifions nom, prénom, poste, ministre, ministère et date de début. Ces informations n'ayant pas de tag, il est nécessaire de mettre des règles de syntaxes afin de les identifier. 


Voici, par exemple, le contenu d'un arrêté de nomination tel que nous l'obtenons via une requête pour un NOR donné :

In [19]:
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Afin de comprendre le format d'un output de nomination, utilisons le NOR d'une nomination donnée
payload = {
    "nor": "ECOP2427670A"
}


# Envoi de la requête
response = requests.post(url, headers=headers, json=payload)

# Vérification et affichage du résultat
if response.status_code == 200:
    data = response.json()
    print(data)  # Affiche le contenu brut de la réponse pour analyse
else:
    print(f"Erreur {response.status_code}: {response.text}")

{'executionTime': 13, 'dereferenced': False, 'id': 'JORFTEXT000050397779_01-01-2999', 'idConteneur': None, 'cid': 'JORFTEXT000050397779', 'title': "Arrêté du 18 octobre 2024 portant nominations au cabinet du ministre de l'économie, des finances et de l'industrie", 'nor': 'ECOP2427670A', 'eli': None, 'alias': None, 'jorfText': 'JORF n°0255 du 26 octobre 2024', 'jurisState': 'Sans état juridique', 'visa': '<p align="left"><br/>Le ministre de l\'économie, des finances et de l\'industrie,<br/>Vu le <a href="/affichTexte.do?cidTexte=JORFTEXT000034938597&categorieLien=cid" title="Décret n°2017-1098 du 14 juin 2017">décret n° 2017-1098 du 14 juin 2017</a> relatif aux collaborateurs du Président de la République et des membres du Gouvernement ;<br/>Vu le <a href="/affichTexte.do?cidTexte=JORFTEXT000050251629&categorieLien=cid" title="Décret n°2024-892 du 23 septembre 2024">décret n° 2024-892 du 23 septembre 2024</a> relatif à la composition des cabinets ministériels ;<br/>Vu le <a href="/affic

Désormais, grâce au module search, nous identifions l'ensemble des nominations de l'année 2024. 

In [23]:
import requests

url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/search/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

payload_template = {
    "recherche": {
        "filtres": [
            {
                "valeurs": ["ARRETE"],
                "facette": "NATURE"
            },
            {
                "dates": {
                    "start": "2024-01-01",
                    "end": "2024-12-31"
                },
                "facette": "DATE_SIGNATURE"
            }
        ],
        "sort": "SIGNATURE_DATE_DESC",
        "fromAdvancedRecherche": False,
        "secondSort": "ID",
        "champs": [
            {
                "criteres": [
                    {"valeur": "cabinet", "operateur": "ET", "typeRecherche": "TOUS_LES_MOTS_DANS_UN_CHAMP"},
                    {"valeur": "ministre", "operateur": "ET", "typeRecherche": "TOUS_LES_MOTS_DANS_UN_CHAMP"}
                ],
                "operateur": "ET",
                "typeChamp": "TITLE"
            }
        ],
        "pageSize": 100,
        "operateur": "ET",
        "typePagination": "DEFAUT",
        "pageNumber": 1
    },
    "fond": "LODA_DATE"
}


# Fetch all pages
all_nors = []
page_number = 1

while True:
    payload_template["recherche"]["pageNumber"] = page_number
    response = requests.post(url, headers=headers, json=payload_template)

    if response.status_code == 200:
        data = response.json()
        results = data.get('results', [])
        
        if not results:  # Break if there are no more results
            break

        # Extract NORs
        nors = [item['nor'] for item in results]
        all_nors.extend(nors)
        
        page_number += 1
    else:
        print("Error during request:", response.status_code, response.text)
        break

print("All NORs for 2024:", all_nors)
print(f"Total NORs retrieved: {len(all_nors)}")

All NORs for 2024: ['PRMX2433989A', 'PRMX2432440A', 'PRMX2432236A', 'PRMX2432079A', 'PRMX2430415A', 'MOMC2426137A', 'MOMC2429204A', 'EAEC2428687A', 'PRMX2429339A', 'MOMC2429076A', 'PRMX2429154A', 'PRMX2428951A', 'PRMX2428841A', 'MOMC2427547A', 'EAEC2427273A', 'PRMX2428095A', 'PRMX2427555A', 'PRMX2427951A', 'PRMX2427731A', 'PRMX2427727A', 'PRMX2427765A', 'PRMX2427371A', 'EAEC2426201A', 'PRMX2427272A', 'PRMX2427372A', 'PRMX2427125A', 'PRMX2426908A', 'PRMX2426706A', 'BCPP2426143A', 'PRMX2426653A', 'PRMX2426171A', 'PRMX2426403A', 'PRMX2426032A', 'PRMX2426008A', 'PRMX2425979A', 'PRMX2425973A', 'PRMX2425839A', 'PRMX2425623A', 'PRMX2425659A', 'ECOP2425558A', 'ECOP2425561A', 'PRMX2425332A', 'PRMX2425424A', 'PRMX2425538A', 'PRMX2424782A', 'PRMX2424503A', 'PRMX2424356A', 'PRMX2423846A', 'PRMX2423727A', 'PRMX2423578A', 'PRMX2423322A', 'PRMX2423250A', 'PRMX2423182A', 'PRMX2422747A', 'PRMX2419839A', 'PRMX2419554A', 'PRMG2215454A', 'PRMX2416563A', 'PRMX2416381A', 'PRMX2416166A', 'PRMX2415349A', 'PRM

Désormais, pour l'ensemble de ces arrêtés publiés au JORF, identifiés par les NOR, nous allons essayer de récupérer les informations souhaitées. Cela n'est pas partie facile : un même arrêté peut avoir une seule ou plusieurs nominations, et la formule employée change selon le ministre qui la rédige, ou, vraisemblablement, sa bonne humeur. 

Nous optons donc pour un regex avec une structure par groupe, en essayant d'être le plus flexible. Par exemple, le genre est donné par la présence de M. ou Mme, et le prénom est le mot (éventuellement composé) qui suit immédiatement M. ou Mme. 

In [9]:
import requests
import pandas as pd
import re
import time

# API details
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",  # il faut faire tourner le premier chunk pour avoir l'access token
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# List of NORs to process

# Regex to extract nominee details
regex = r"(M\.|Mme) ([\w\-]+) ([\w\-]+(?: [\w\-]+)?)(?:,| est nommé(e)?) (.*?)(?:,|:|;|\s)?(?: à compter du ([\d]{1,2} [^\d]+ [\d]{4}|\d{1,2}/\d{1,2}/\d{4}))(?:,|:|;|\n)?"
#r"(M\.|Mme) ([\w\-]+) ([\w\-]+(?: [\w\-]+)?)(?:,| est nommée?| est nommée?) (.*?)(?:,|:|;|\s)?(?: à compter du ([\d]{1,2} [^\d]+ [\d]{4}|\d{1,2}/\d{1,2}/\d{4}))(?:,|:|;|\n)?"

# Data collection
all_nominees = []

# Loop through each NOR
for nor in all_nors:
    payload = {"nor": nor}
    try:
        # Send the request
        response = requests.post(url, headers=headers, json=payload)

        if response.status_code == 200:
            data = response.json()

            # Extract details for each article
            articles = data.get("articles", [])
            for article in articles:
                content = article.get("content", "")

                # Apply regex to find matches
                matches = re.findall(regex, content, re.IGNORECASE | re.DOTALL)

                for match in matches:
                    genre, prenom, nom, _, titre, date = match
                    all_nominees.append({
                        "Genre": genre,
                        "Prénom": prenom,
                        "Nom": nom,
                        "Titre": titre.strip(),
                        "Date de début": date.strip() if date else "Non précisée",
                        "NOR": nor,
                        "Ministre": re.search(r"Fait le .*?<br/>(.*?)</p>", data.get("signers", ""), re.DOTALL).group(1).strip() if "signers" in data else "Non spécifié",
                        "Ministère": re.search(r"cabinet du (.*)|cabinet de la (.*)", data.get("title", ""), re.IGNORECASE).group(1) or re.search(r"cabinet du (.*)|cabinet de la (.*)", data.get("title", ""), re.IGNORECASE).group(2) if re.search(r"cabinet du (.*)|cabinet de la (.*)", data.get("title", ""), re.IGNORECASE) else "Non spécifié"
                    })
        elif response.status_code == 429:
            # Handle rate-limiting
            retry_after = int(response.headers.get("Retry-After", 5))
            print(f"Rate limit hit. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
            continue
        else:
            print(f"Error {response.status_code} for NOR {nor}: {response.text}")
    except Exception as e:
        print(f"Exception occurred for NOR {nor}: {e}")

    # Avoid hitting the API rate limit
    time.sleep(0.3)  # Adjust sleep time based on your successful 429 handling

# Convert to DataFrame
df = pd.DataFrame(all_nominees)

# Display the resulting DataFrame
print(df)

   Genre      Prénom             Nom  \
0    Mme      Sabine         HAMMOND   
1    Mme  Marie-Anne        LAVERGNE   
2    Mme   Madeleine           ROUOT   
3    Mme       Anais  KEITA-GORISSEN   
4    Mme       Sarah           ALLIX   
..   ...         ...             ...   
86   Mme      Samira           Jemaï   
87   Mme       Chloé          Muller   
88   Mme      Margot          Provot   
89    M.      Jérémy        HERROUIN   
90    M.      Amaury           DUMAY   

                                                Titre     Date de début  \
0   conseillère technique en charge des discours (...   2 décembre 2024   
1   conseillère services financiers et économie in...  24 novembre 2024   
2   conseillère technique en charge de l'argumentaire  12 novembre 2024   
3                        conseillère technique presse  28 novembre 2024   
4   conseillère territoires et services publics au...  18 novembre 2024   
..                                                ...               .

**Erreurs parfois rencontrées** 
- Erreur 401 : unauthorized, vérifier que son token n'est pas périmé, sinon c'est sans doute les requêtes provenant des serveurs du SSP Cloud qui sont bloquées. Résolution : passer en local.
- Erreur 429 : too many requests, modifier le temps entre chaque requête

Si le script s'execute sans erreur (souvent en une vingtaine de secondes), une dernière étape de nettoyage est nécessaire.

In [ ]:
import re

## 1. Nettoyage
def clean_titre_column(titre):
    # Remove HTML elements
    titre = re.sub(r'<.*?>', '', titre)
    # Remove elements starting with 'à comp' and ending with '2024'
    titre = re.sub(r'à comp.*?2024', '', titre, flags=re.DOTALL)
    # Remove semicolons, colons, and commas
    titre = re.sub(r'[;:,.]', '', titre)
    # Strip extra whitespace
    return titre.strip()

# Apply the cleaning function to the 'Titre' column
df['Titre'] = df['Titre'].apply(clean_titre_column)

## 2. Enregistrement en csv
df.to_csv('nominations_2024.csv', index=False)



In [ ]:
import s3fs
## 3. Envoi au bucket
# Configurons la connection
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

# Détails de notre bucket et fichiers
MY_BUCKET = "sachanassensae"
fs.ls(MY_BUCKET)
LOCAL_FILE = "nominations_2024.csv"

# Enregistrons
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/nominations_2024.csv"
fs.put(LOCAL_FILE, FILE_PATH_OUT_S3)


[None]

In [17]:
# Commandes afin de voir les NOR et le token utilisés
print(access_token)
print(all_nors)

# Vérifions la présence du bucket
fs.ls(f"{MY_BUCKET}/diffusion")

Nusnx5r1ilcU3ZULWbZMamwZjKMEQpDQcWvqNTxib2nL7E7syxQ0d7
['PRMX2433989A', 'PRMX2432440A', 'PRMX2432236A', 'PRMX2432079A', 'PRMX2430415A', 'MOMC2426137A', 'MOMC2429204A', 'EAEC2428687A', 'PRMX2429339A', 'MOMC2429076A', 'PRMX2429154A', 'PRMX2428951A', 'PRMX2428841A', 'MOMC2427547A', 'EAEC2427273A', 'PRMX2428095A', 'PRMX2427555A', 'PRMX2427951A', 'PRMX2427731A', 'PRMX2427727A', 'PRMX2427765A', 'PRMX2427371A', 'EAEC2426201A', 'PRMX2427272A', 'PRMX2427372A', 'PRMX2427125A', 'PRMX2426908A', 'PRMX2426706A', 'BCPP2426143A', 'PRMX2426653A', 'PRMX2426171A', 'PRMX2426403A', 'PRMX2426032A', 'PRMX2426008A', 'PRMX2425979A', 'PRMX2425973A', 'PRMX2425839A', 'PRMX2425623A', 'PRMX2425659A', 'ECOP2425558A', 'ECOP2425561A', 'PRMX2425332A', 'PRMX2425424A', 'PRMX2425538A', 'PRMX2424782A', 'PRMX2424503A', 'PRMX2424356A', 'PRMX2423846A', 'PRMX2423727A', 'PRMX2423578A', 'PRMX2423322A', 'PRMX2423250A', 'PRMX2423182A', 'PRMX2422747A', 'PRMX2419839A', 'PRMX2419554A', 'PRMG2215454A', 'PRMX2416563A', 'PRMX2416381A', 

['sachanassensae/diffusion/.keep',
 'sachanassensae/diffusion/nominations_2024.csv']

Ce nettoyage est, dans une certaine mesure, un cache-misère : on perd un certain nombre de nominations, particulièrement lorsque les arrêtés de nominations comportent un grand nombre de prénoms, répartis par pôle. C'est par exemple le cas dans cet arrêté [https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000049010626].